# LangChain and Google GenAI Integration Project

This script demonstrates the integration of LangChain with Google Generative AI,
embedding-based similarity search, and RAG (Retrieval-Augmented Generation) techniques.


In [20]:
!pip install langchain-community langchain_chroma wikipedia
!pip install google-generativeai

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=c8a26a82a482f9bf18b127235960179cb13c550d9e4fb12021aa5d5549833f7d
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [53]:
# Import necessary packages
import os
import google.generativeai as genai
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.document_loaders import WikipediaLoader
from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.load import dumps, loads
from langchain.schema.output_parser import StrOutputParser
from langchain_core.output_parsers import BaseOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from pydantic import BaseModel
from typing import Any, Dict, Iterator, List, Mapping, Optional


## Section 1: Google Generative AI Configuration
This section configures the Google Generative AI model and demonstrates a basic prompt-response flow.

In [12]:
genai.configure(api_key="your_key")  # Replace with your actual API key

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("The opposite of hot is")
print(f"Response from Google GenAI: {response.text}")

Response from Google GenAI: cold



## Section 2: Custom LLM Implementation
This section implements a custom LLM class for LangChain compatibility.


In [13]:
class CustomLLM(LLM):
    """Custom implementation of an LLM to integrate with Google GenAI."""

    def _call(
        self, prompt: str, stop: Optional[List[str]] = None, run_manager: Optional[CallbackManagerForLLMRun] = None, **kwargs: Any
    ) -> str:
        """Generate text from the given prompt using Google GenAI."""
        model = genai.GenerativeModel('gemini-1.5-flash')
        response = model.generate_content(prompt)
        return response.text

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model_name": "Gemini_1.5_Flash"}

    @property
    def _llm_type(self) -> str:
        return "custom"

# Initialize and test the CustomLLM
llm = CustomLLM()
print(f"Custom LLM Initialized: {llm}")
print(f"LLM Test Invocation: {llm.invoke('This is a test message.')}")


Custom LLM Initialized: CustomLLM
Params: {'model_name': 'Gemini_1.5_Flash'}
LLM Test Invocation: Okay, I received your test message.  How can I help you?



## Section 3: Semantic Similarity Search with Sentence Embeddings
This section uses `sentence-transformers` to find semantically similar documents.


In [14]:
# Load the sentence embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Example query and documents
query = "Where is XYZ having its head office?"
documents = [
    "XYZ's head office is located in New York City.",
    "There is an office for XYZ in London.",
    "The head office of XYZ is in San Francisco."
]

# Compute embeddings and calculate cosine similarity
query_embedding = embedding_model.encode(query)
document_embeddings = embedding_model.encode(documents)
similarities = cosine_similarity([query_embedding], document_embeddings)[0]
most_similar_index = similarities.argmax()

print(f"Most Similar Document: {documents[most_similar_index]}")
print(f"Similarity Score: {similarities[most_similar_index]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Most Similar Document: XYZ's head office is located in New York City.
Similarity Score: 0.8595919013023376


## Section 4: Retrieval-Augmented Generation (RAG)
Combining LangChain, Chroma, and Google GenAI for contextual document retrieval and generation.

In [55]:
# Load documents using WikipediaLoader
docs = WikipediaLoader(query="family guy tv series", load_max_docs=5).load()

# Split documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=3500, chunk_overlap=100, separator=" ")
split_docs = text_splitter.split_documents(docs)

# Initialize Chroma for document embeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(split_docs, embedding=embedding_function)  # Pass the embedding_function object, not its encode method
# Retrieve relevant documents and generate text
retriever = db.as_retriever(k=7)
prompt = PromptTemplate.from_template(
    "Answer the following question based on the given context: {context}, question: {question}"
)

rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(doc.page_content for doc in docs)), "question": RunnablePassthrough()}
    | prompt
    | RunnableLambda(func=lambda x: model.generate_content(x.text).text)
    | StrOutputParser()
)

# Test RAG pipeline
question = "Who is the voice actor for Meg in Family Guy?"
answer = rag_chain.invoke(question)
print(f"RAG Answer: {answer}")

RAG Answer: Lacey Chabert voiced Meg Griffin in the first nine episodes of Family Guy.  Mila Kunis took over the role from episode 10 onwards.



## Section 5: Multi-Query Fusion for Better Document Retrieval
Using multiple rephrasings of a question to retrieve the most relevant documents.


In [56]:
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for multiple query generation."""
    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))[1:]

output_parser = LineListOutputParser()
query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Generate multiple versions of the given question for better retrieval: {question}"""
)

llm_chain = query_prompt | llm | output_parser
retriever = MultiQueryRetriever(
    retriever=db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)

# Test Multi-Query Fusion
questions = llm_chain.invoke({"question": question})
print(f"Rephrased Questions: {questions}")

Rephrased Questions: ['**Variations focusing on the character:**', '* What actor voices Meg Griffin in Family Guy?', '* Who plays Meg in Family Guy? (Implies voice acting)', '* Who is the voice of Meg Griffin on Family Guy?', '* Family Guy: Who voices Meg?', '* Who provides the voice for Meg Griffin? (More formal)', "* Who's the voice actress for Meg in Family Guy? (Specifies gender, though Mila Kunis is the current voice actress, and previous voice actress Lacey Chabert was also female)", '**Variations focusing on the actor:**', '* Who voices Meg Griffin Family Guy? (Removes unnecessary word)', "* Who is Mila Kunis's Family Guy character? (Assumes knowledge of the voice actress)", '* Which Family Guy character does Mila Kunis voice? (Similar to above)', '* Is Mila Kunis the voice of Meg Griffin? (Yes/No question, good for filtering)', '* Who was the original voice actor for Meg Griffin in Family Guy? (Addresses the change in voice actors)', '**Variations using keywords:**', '* Family 

## Section 6: RAG Fusion Implementation
combining ranked retrieval results
from multiple sources. The RRF algorithm aggregates ranked lists to enhance the overall
retrieval effectiveness. This is particularly useful in systems using multi-query retrieval.

In [57]:
def reciprocal_rank_fusion(results: list[list], k=6):
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    # Iterate through each document in the list, with its rank (position in the list)
    for rank, doc in enumerate(results):
        # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
        doc_str = dumps(doc)
        # print(doc_str)
        # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
        if doc_str not in fused_scores:
            fused_scores[doc_str] = 0
        # Retrieve the current score of the document, if any
        previous_score = fused_scores[doc_str]
        # Update the score of the document using the RRF formula: 1 / (rank + k)
        fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        loads(doc)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results
unique_docs = retriever.invoke(question)
final_docs = reciprocal_rank_fusion(unique_docs)

In [59]:
rank_lambda = RunnableLambda(lambda x: reciprocal_rank_fusion(x)[:2])
# rank_lambda.invoke(unique_docs)

# add the ranking to llm_chain
llm_chain = QUERY_PROMPT | llm | output_parser | rank_lambda
retriever = MultiQueryRetriever(
    retriever=db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)
rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(doc.page_content for doc in docs)), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = 'Who is actor for Meg in family Guy in each season of the series?'
rag_chain.invoke(question)


'Mila Kunis has voiced Meg Griffin in *Family Guy* since Season 2.  Before that, Lacey Chabert voiced Meg in Season 1.\n'